In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Data Preprocessing

In [ ]:
# Imports for pytorch
import numpy as np
import torch
import torchvision
from torch import nn
import matplotlib
from matplotlib import pyplot as plt
import tqdm
import copy
import torch.utils.data as data

SEED = 1234

torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [ ]:
transform = torchvision.transforms.ToTensor()

unprocessed_train_data = torchvision.datasets.CIFAR100(
    root="data",
    train=True,
    download=True
)



arr_mean = np.mean(unprocessed_train_data.data, axis=(0, 1, 2)) / 255
arr_sd = np.std(unprocessed_train_data.data, axis=(0, 1, 2)) / 255

SIZE = 112


train_transforms = torchvision.transforms.Compose([
                           torchvision.transforms.Resize(SIZE),
                           torchvision.transforms.RandomRotation(5),
                           torchvision.transforms.RandomHorizontalFlip(0.5),
                           torchvision.transforms.RandomCrop(SIZE, padding=10),
                           torchvision.transforms.ToTensor(),
                           torchvision.transforms.Normalize(mean=arr_mean,
                                                std=arr_sd)
                       ])

test_transforms = torchvision.transforms.Compose([
                           torchvision.transforms.Resize(SIZE),
                           torchvision.transforms.ToTensor(),
                           torchvision.transforms.Normalize(mean=arr_mean,
                                                std=arr_sd)
                           ])
training_data = torchvision.datasets.CIFAR10(root = 'data',
                              train=True,
                              download=True,
                              transform=train_transforms)

test_data = torchvision.datasets.CIFAR10(root = 'data',
                             train=False,
                             download=True,
                             transform=test_transforms)

In [ ]:
test_data

In [ ]:
train_data, val_data = data.random_split(training_data,
                                           [int(training_data.data.shape[0] * 0.9), int(training_data.data.shape[0] * 0.1)])


In [ ]:
val_data = copy.deepcopy(val_data)
val_data.dataset.transform = test_transforms #make sure the val data uses test transform

# **Build Model Architecture**


In [ ]:
class Classifier(nn.Module):
    def __init__(self, output_dim):
        super().__init__()

        self.linear1 = nn.Linear(25088, 256)
        #self.linear2 = nn.Linear(512, 256)
        self.linear2 = nn.Linear(256, 256)
        self.linear3 = nn.Linear(512, 256)
        self.linear4 = nn.Linear(512, 256)
        #self.linear5 = nn.Linear(512, 256)
        #self.linear6 = nn.Linear(1024, 512)
        #self.linear7 = nn.Linear(512, 256)
        
        self.linearOutput = nn.Linear(512, output_dim)

        self.dropout = nn.Dropout(0.3)
        
        self.layer1 = nn.LayerNorm(256)
        #self.layer2 = nn.LayerNorm(256)
        

    def forward(self, x):
        
        linearOutput1 = self.linear1(x)
        x = nn.functional.relu(linearOutput1)
        x = self.layer1(x)
        x = self.dropout(x)

        linearOutput2 = self.linear2(x)
        x = nn.functional.relu(linearOutput2)
        x = torch.cat([x, linearOutput2], -1)
        x = self.dropout(x)

        
        linearOutput3 = self.linear3(x)
        x = nn.functional.relu(linearOutput3)
        x = torch.cat([x, linearOutput3], -1)
        x = self.dropout(x)

        linearOutput4 = self.linear4(x)
        x = nn.functional.relu(linearOutput4)
        x = torch.cat([x, linearOutput4], -1)
        x = self.dropout(x)

        x = self.linearOutput(x)
        
        return x
        


        




In [ ]:
class VGG(nn.Module):
    def __init__(self, features, output_dim):
        super().__init__()

        self.features = features

        self.avgpool = nn.AdaptiveAvgPool2d((7,7))

        self.classifier = Classifier(output_dim)
    

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        h = x.view(x.shape[0], -1)
        x = self.classifier(h)
        return x, h

 
                
def design_layers(config, batch_norm):

    layers = []
    in_channels = 3

    for c in config:

        if c == 'M':
            layers += [nn.MaxPool2d(kernel_size=2, stride = 2)]
        else:
        
            conv2d = nn.Conv2d(in_channels, c, kernel_size=3, padding=1)
            if batch_norm:
                layers += [conv2d, nn.BatchNorm2d(c), nn.ReLU(inplace=True)]
            else:
                layers += [conv2d, nn.ReLU(inplace=True)]
            in_channels = c

    return nn.Sequential(*layers)
seq = [512, 512, "M", 512, 512, "M", 256, 256, "M", 64, "M"]
#[64, "M", 128, 128, "M", 256, 256, "M", 512, 512, "M", 512, 512, "M"]
#[64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 512, 512, 512, 'M', 512, 512, 512, 'M']



# reference from https://pytorch.org/vision/0.8/_modules/torchvision/models/vgg.html

In [ ]:
vgg1_layers = design_layers(seq, batch_norm=True)
model = VGG(vgg1_layers, 32)
vgg1_layers

In [ ]:
BATCH_SIZE = 128

all_train_iterator = data.DataLoader(training_data,
                                 shuffle=True,
                                 batch_size=BATCH_SIZE)


train_iterator = data.DataLoader(train_data,
                                 shuffle=True,
                                 batch_size=BATCH_SIZE)

val_iterator = data.DataLoader(val_data,
                                 batch_size=BATCH_SIZE)



In [ ]:
all_train_iterator

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


model = model.to(device)
CEL = nn.CrossEntropyLoss().to(device)

In [ ]:
device

In [ ]:
import torch.optim as optim


params = [
          {'params': model.features.parameters()}, #, 'lr': 5e-4 / 10},
          {'params': model.classifier.parameters()}
         ]

optimizer = optim.AdamW(params, lr=5e-3, weight_decay = 1e-4)
from tqdm.notebook import trange, tqdm



In [ ]:
from tqdm.notebook import trange, tqdm

def train(model, iterator, optimizer, CEL, device):

    epoch_loss = 0
    epoch_accuracy = 0

    model.train()

    for (x, y) in tqdm(iterator, desc="Training", leave=False):

        x = x.to(device)
        y = y.to(device)

        optimizer.zero_grad()

        y_pred, _ = model(x)

        loss = CEL(y_pred, y)

        accuracy = calculate_accuracy(y_pred, y)

        loss.backward()

        optimizer.step()

        epoch_loss += loss.item()
        epoch_accuracy += accuracy.item()

    return epoch_loss / len(iterator), epoch_accuracy / len(iterator)


def evaluate(model, iterator, CEL, device):

    epoch_loss = 0
    epoch_accuracy = 0

    model.eval()

    with torch.no_grad():

        for (x, y) in tqdm(iterator, desc="Evaluating", leave=False):

            x = x.to(device)
            y = y.to(device)

            y_pred, _ = model(x)

            loss = CEL(y_pred, y)

            accuracy = calculate_accuracy(y_pred, y)

            epoch_loss += loss.item()
            epoch_accuracy += accuracy.item()

    return epoch_loss / len(iterator), epoch_accuracy / len(iterator)

def calculate_accuracy(y_pred, y):
    first_pred = y_pred.argmax(1, keepdim=True)
    correct = first_pred.eq(y.view_as(first_pred)).sum()
    accuracy = correct.float() / y.shape[0]
    return accuracy

In [ ]:
EPOCHS = 20

best_val_loss = float('inf')

train_accuracies = []
val_accuracies = []
train_losses = []
val_losses = []

for epoch in trange(EPOCHS):

    train_loss, train_accuracy = train(model, train_iterator, optimizer, CEL, device)
    val_loss, val_accuracy = evaluate(model, val_iterator, CEL, device)

    train_accuracies.append(train_accuracy)
    val_accuracies.append(val_accuracy)
    train_losses.append(train_loss)
    val_losses.append(val_loss)

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), 'tut1-model.pt')

    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_accuracy*100:.2f}%')
    print(f'\t Val. Loss: {val_loss:.3f} |  Val. Acc: {val_accuracy*100:.2f}%')


plt.plot(range(1, EPOCHS + 1), train_accuracies, label='Train Accuracy')
plt.plot(range(1, EPOCHS + 1), val_accuracies, label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Train/Validation Accuracy')
plt.legend()
plt.show()

plt.plot(range(1, EPOCHS + 1), train_losses, label='Train Loss')
plt.plot(range(1, EPOCHS + 1), val_losses, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Train/Validation Loss')
plt.legend()
plt.show()

In [ ]:
from PIL import Image
import os

class CIFAR10Test(torchvision.datasets.VisionDataset):

    def __init__(self, transform=None, target_transform=None):
        super(CIFAR10Test, self).__init__(None, transform=transform,
                                      target_transform=target_transform)
        assert os.path.exists("/kaggle/input/cifar10-test-data-sp24-npy/cifar10_test_data_sp24.npy"), "You must upload the test data to the file system."
        self.data = [np.load("/kaggle/input/cifar10-test-data-sp24-npy/cifar10_test_data_sp24.npy", allow_pickle=False)]

        self.data = np.vstack(self.data).reshape(-1, 3, 32, 32)
        self.data = self.data.transpose((0, 2, 3, 1))  # convert to HWC

    def __getitem__(self, index: int):
        img = self.data[index]
        img = Image.fromarray(img)
        if self.transform is not None:
            img = self.transform(img)
        return img

    def __len__(self) -> int:
        return len(self.data)

# Create the test dataset
testing_data = CIFAR10Test(
    transform=test_transforms # NOTE: Make sure transform is the same as used in the training dataset.
)

test_data = torchvision.datasets.CIFAR10(root = 'data',
                             train=False,
                             download=True,
                             transform=test_transforms)

In [ ]:
test_iterator = data.DataLoader(testing_data,
                                batch_size=BATCH_SIZE,
                               shuffle = False)


In [ ]:
model

In [ ]:


def get_predictions(model, iterator):

    model.eval()
    model = model.to(device)
    labels = []

    with torch.no_grad():
        for x in tqdm(iterator):
            x = x.to(device)

            y_pred, _ = model(x)
            
            _, predicted_labels = torch.max(y_pred, 1)
            
            labels.extend(predicted_labels.tolist())
    

    return labels


predictions = get_predictions(model, test_iterator)
len(predictions)

In [ ]:
import pandas as pd

if isinstance(predictions, np.ndarray):
    predictions = predictions.astype(int)
else:
    predictions = np.array(predictions, dtype=int)
assert predictions.shape == (len(testing_data),), "Predictions were not the correct shape"
df = pd.DataFrame({'Category': predictions})
df.index += 1  # Ensures that the index starts at 1.
df.to_csv('submission.csv', index_label='Id')

In [ ]:
def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict
unpickle()